In [1]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRunSup.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
]

values = [1, 2, 3]

df["fraction"] = np.select(conditions, values)


dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0"])
dfPrepSample.head()


,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
0,66,25310,2625,24.0,149.0,808,1
1,46,28154,5825,96.0,174.0,836,2
2,45,7361,1400,9.0,74.0,390,3
3,34,8488,2125,26.5,74.0,390,3
4,27,4589,550,6.0,46.0,261,1


In [2]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
1100,44,11209,1000,11.0,86.0,549,1
2100,24,10508,2775,27.5,60.0,371,3
1066,21,6471,1625,22.5,50.0,382,3
2887,18,3658,575,4.5,36.0,235,3
2363,47,12998,1775,29.0,94.0,523,2


In [3]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
2544,31,9884,1825,14.5,60.0,466,3
667,68,25812,1250,11.0,149.0,682,1
2404,13,80,0,0.0,15.0,18,2
221,56,32749,10750,145.0,208.0,951,3
2428,39,19025,725,8.0,70.0,517,1


In [4]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
1100,44,11209,1000,11.0,86.0,549,1
2100,24,10508,2775,27.5,60.0,371,3
1066,21,6471,1625,22.5,50.0,382,3
2887,18,3658,575,4.5,36.0,235,3
2363,47,12998,1775,29.0,94.0,523,2
...,...,...,...,...,...,...,...
147,54,32935,4300,40.0,106.0,710,3
2599,33,5307,1225,11.5,52.0,335,3
3071,49,17398,4700,58.0,146.0,718,3
2199,49,10664,800,9.0,86.0,481,1


In [5]:

dfTrainLabels = dfTrainFeatures.pop("total_army_value")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[   44. , 11209. ,    11. ,    86. ,   549. ,     1. ],
       [   24. , 10508. ,    27.5,    60. ,   371. ,     3. ],
       [   21. ,  6471. ,    22.5,    50. ,   382. ,     3. ],
       ...,
       [   49. , 17398. ,    58. ,   146. ,   718. ,     3. ],
       [   49. , 10664. ,     9. ,    86. ,   481. ,     1. ],
       [   52. , 17586. ,    21. ,    94. ,   528. ,     1. ]])

In [6]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [7]:
sc_model = tf.keras.Sequential([layers.Dense(30, activation="relu"),layers.Dense(10, activation="relu"), layers.Dense(1)])

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [8]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=350)

Epoch 1/350
84/84 [==============================] - 0s 663us/step - loss: 39132112.0000
Epoch 2/350
84/84 [==============================] - 0s 637us/step - loss: 14256556.0000
Epoch 3/350
84/84 [==============================] - 0s 654us/step - loss: 9242412.0000
Epoch 4/350
84/84 [==============================] - 0s 643us/step - loss: 10589469.0000
Epoch 5/350
84/84 [==============================] - 0s 626us/step - loss: 8764767.0000
Epoch 6/350
84/84 [==============================] - 0s 622us/step - loss: 8189550.5000
Epoch 7/350
84/84 [==============================] - 0s 612us/step - loss: 9184734.0000
Epoch 8/350
84/84 [==============================] - 0s 629us/step - loss: 7576720.0000
Epoch 9/350
84/84 [==============================] - 0s 589us/step - loss: 6404670.5000
Epoch 10/350
84/84 [==============================] - 0s 609us/step - loss: 6385626.5000
Epoch 11/350
84/84 [==============================] - 0s 594us/step - loss: 5869511.5000
Epoch 12/350
84/84 [=======

In [9]:
dfTestFeatures = dfTestSample.copy()

dfTestFeatures

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
2544,31,9884,1825,14.5,60.0,466,3
667,68,25812,1250,11.0,149.0,682,1
2404,13,80,0,0.0,15.0,18,2
221,56,32749,10750,145.0,208.0,951,3
2428,39,19025,725,8.0,70.0,517,1
...,...,...,...,...,...,...,...
3276,61,14599,3450,28.0,114.0,603,3
1128,36,7504,4600,55.0,132.0,485,3
1654,22,10911,1825,23.5,52.0,449,3
418,46,7015,1500,9.0,82.0,409,3


In [10]:

dfTestLabels = dfTestFeatures.pop("total_army_value")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

2544     1825
667      1250
2404        0
221     10750
2428      725
Name: total_army_value, dtype: int64

In [11]:
from sklearn.metrics import r2_score

predictedValues = sc_model.predict(dfTestFeatures)

21/21 [==============================] - 0s 600us/step


In [12]:

print(f"The R^2 value in the test set is {r2_score(dfTestLabels, predictedValues)}")

The R^2 value in the test set is 0.9320898601937243


In [13]:
# optimizer with scaling and lin reg

import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr

data = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')
data = data[1350:]


# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfOpt = dfPrepSample.drop(columns=[
                               "total_workers"])

dfOpt.head

<bound method NDFrame.head of       resource_mining  total_army_value  total_army  supply  time  fraction
0               25310              2625        24.0   149.0   808         1
1               28154              5825        96.0   174.0   836         2
2                7361              1400         9.0    74.0   390         3
3                8488              2125        26.5    74.0   390         3
4                4589               550         6.0    46.0   261         1
...               ...               ...         ...     ...   ...       ...
3323             7872              1750        25.0    70.0   401         2
3324             6490               600         8.0    39.0   261         1
3325             4774               600         8.0    39.0   251         1
3326            13131               475         5.0    70.0   429         1
3327             9132              3625        35.5    90.0   458         3

[3328 rows x 6 columns]>

In [14]:


# create variables
# create Model

m = gp.Model("Worker optimizer")
gppd.set_interactive()

w = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_workers", vtype=gp.GRB.INTEGER)
av = gppd.add_vars(m, pandas_obj=dfOpt,
                      name="total_army_value")

a = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_army", vtype=gp.GRB.INTEGER)
s = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="supply", vtype=gp.GRB.INTEGER)


m.update()
dfOpt = dfOpt.drop(columns=["total_army_value"])

feats = dfOpt
feats = pd.concat([w, feats], axis=1)
feats.head

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-17


<bound method NDFrame.head of                          total_workers  resource_mining  total_army  supply  \
0        <gurobi.Var total_workers[0]>            25310        24.0   149.0   
1        <gurobi.Var total_workers[1]>            28154        96.0   174.0   
2        <gurobi.Var total_workers[2]>             7361         9.0    74.0   
3        <gurobi.Var total_workers[3]>             8488        26.5    74.0   
4        <gurobi.Var total_workers[4]>             4589         6.0    46.0   
...                                ...              ...         ...     ...   
3323  <gurobi.Var total_workers[3323]>             7872        25.0    70.0   
3324  <gurobi.Var total_workers[3324]>             6490         8.0    39.0   
3325  <gurobi.Var total_workers[3325]>             4774         8.0    39.0   
3326  <gurobi.Var total_workers[3326]>            13131         5.0    70.0   
3327  <gurobi.Var total_workers[3327]>             9132        35.5    90.0   

      time  fraction 

In [15]:


m.setObjective(av.sum(), gp.GRB.MAXIMIZE) # Zielfunktion
"""
gppd.add_constrs(m, ((av - 50*w)/100), gp.GRB.GREATER_EQUAL, a) # Gleichung 2
"""
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Gleichung 3
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, s) # Gleichung 4
m.update()

pred_constr = add_predictor_constr(m, sc_model, feats, av) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.optimize()


Model for sequential1:
282880 variables
136448 constraints
133120 general constraints
Input has shape (3328, 6)
Output has shape (3328, 1)

--------------------------------------------------------------------------------
Layer           Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
dense0            (3328, 30)       199680        99840            0        99840 (relu)

dense1            (3328, 10)        66560        33280            0        33280 (relu)

dense2             (3328, 1)            0         3328            0            0

--------------------------------------------------------------------------------
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 143104 rows, 296192

In [ ]:

outputs = pred_constr.input_values
pred_constr.input_values



GurobiError: Unable to retrieve attribute 'X'